In [ ]:
from abc import ABC, abstractmethod

# ---- ITEM SYSTEM ----
class Item:
    """Base class for all items."""
    def __init__(self, name, effect, stat, value, slot=None):
        self.name = name
        self.effect = effect  # 'increase' or 'restore'
        self.stat = stat  # 'health', 'mana', 'strength', etc.
        self.value = value  # How much it affects the stat
        self.slot = slot  # Equipment slot (weapon, chest, head, etc.)

    def apply(self, character):
        """Applies item effects to a character."""
        if self.effect == "increase":
            if self.stat == "health":
                character.base_health += self.value
            elif self.stat == "mana":
                character.base_mana += self.value
            elif self.stat == "strength":
                character.strength += self.value
            elif self.stat == "agility":
                character.agility += self.value
            elif self.stat == "intellect":
                character.intellect += self.value
        elif self.effect == "restore":
            if self.stat == "health":
                character.base_health = min(character.base_health + self.value, character.max_health)
            elif self.stat == "mana":
                character.base_mana = min(character.base_mana + self.value, character.max_mana)
        return f"{character.name} used {self.name} → {self.stat} +{self.value}!"

class Armor(Item):
    """Armor increases defense and agility"""
    def __init__(self, name, defense, agility, slot):
        super().__init__(name, "increase", None, 0, slot)
        self.defense = defense
        self.agility = agility

class Weapon(Item):
    """Weapons increase strength or magic power"""
    def __init__(self, name, damage, stat_boost, slot="weapon"):
        super().__init__(name, "increase", stat_boost, damage, slot)

# ---- SPELL SYSTEM ----
class Spell:
    """Defines spells that consume mana."""
    def __init__(self, name, damage, mana_cost, level_required):
        self.name = name
        self.damage = damage
        self.mana_cost = mana_cost
        self.level_required = level_required

    def cast(self, character):
        """Casts a spell if the character has enough mana."""
        if character.level < self.level_required:
            return f"{character.name} is not high enough level to cast {self.name}!"
        if character.base_mana < self.mana_cost:
            return f"Not enough mana to cast {self.name}!"
        character.base_mana -= self.mana_cost
        return f"{character.name} cast {self.name}, dealing {self.damage} damage!"

# ---- ABSTRACT CLASSES ----
class Race(ABC):
    """Abstract class for races with base stats."""
    def __init__(self, name, **kwargs):
        self.name = name
        self.strength = kwargs.get("strength", 10)
        self.agility = kwargs.get("agility", 10)
        self.intellect = kwargs.get("intellect", 10)
        self.defense = kwargs.get("defense", 5)

    @abstractmethod
    def racial_bonus(self):
        pass

class ClassType(ABC):
    """Abstract class for character classes."""
    def __init__(self, name, **kwargs):
        self.name = name
        self.base_health = kwargs.get("base_health", 100)
        self.base_mana = kwargs.get("base_mana", 50)
        self.abilities = {}  # Level-based abilities
        self.spells = []  # Available spells

    @abstractmethod
    def special_ability(self, level):
        """Returns the ability unlocked at a given level."""
        return self.abilities.get(level, "No new abilities at this level.")

# ---- EQUIPMENT MANAGER ----
class EquipmentManager:
    """Handles equipping and unequipping items."""
    def __init__(self):
        self.equipped_items = {
            "head": None,
            "chest": None,
            "legs": None,
            "weapon": None,
        }

    def equip_item(self, item):
        """Equips an item if the slot is available."""
        if item.slot in self.equipped_items:
            self.equipped_items[item.slot] = item
            return f"Equipped {item.name} in {item.slot} slot!"
        return "Cannot equip this item."

    def unequip_item(self, slot):
        """Unequips an item from a given slot."""
        if slot in self.equipped_items and self.equipped_items[slot]:
            removed_item = self.equipped_items[slot]
            self.equipped_items[slot] = None
            return f"Unequipped {removed_item.name} from {slot} slot!"
        return "No item equipped in this slot."

# ---- CLASS TYPE IMPLEMENTATIONS ----
class Warrior(ClassType):
    def __init__(self):
        super().__init__("Warrior", base_health=150, base_mana=30)
        self.abilities = {1: "Charge - Rush and deal damage!"}
        self.spells = [Spell("Battle Shout", 0, 10, 2)]

class Mage(ClassType):
    def __init__(self):
        super().__init__("Mage", base_health=80, base_mana=150)
        self.abilities = {1: "Fireball - Cast a fire spell!"}
        self.spells = [
            Spell("Fireball", 30, 20, 1),
            Spell("Ice Barrier", 0, 40, 5),
            Spell("Teleport", 0, 50, 10),
        ]

class Rogue(ClassType):
    def __init__(self):
        super().__init__("Rogue", base_health=110, base_mana=80)
        self.abilities = {1: "Stealth - Become invisible!"}
        self.spells = [Spell("Shadowstep", 20, 15, 3)]

# ---- CHARACTER CLASS ----
class Character(Race, ClassType, EquipmentManager):
    def __init__(self, name, race, class_type):
        Race.__init__(self, race.name)
        ClassType.__init__(self, class_type.name)
        EquipmentManager.__init__(self)
        self.name = name
        self.level = 1
        self.experience = 0
        self.inventory = []

    def gain_experience(self, xp):
        self.experience += xp
        if self.experience >= self.level * 100:
            self.level_up()

    def level_up(self):
        self.level += 1
        self.base_health += 10
        self.base_mana += 5
        self.strength += 2
        self.agility += 2
        self.intellect += 2
        print(f"🎉 {self.name} leveled up to Level {self.level}!")

    def cast_spell(self, spell_name):
        for spell in self.spells:
            if spell.name == spell_name:
                return spell.cast(self)
        return f"{self.name} does not know the spell {spell_name}."

# ---- TEST EXAMPLE ----
if __name__ == "__main__":
    hero = Character("Jaina", Human(), Mage())
    hero.gain_experience(150)  # Level up
    print(hero.cast_spell("Fireball"))  # Should work
    print(hero.cast_spell("Teleport"))  # Should fail (not high enough level)
